In [1]:
import pandahouse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.power import TTestIndPower
from scipy import stats
import scipy
import math
from statsmodels.stats.power import TTestIndPower
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
%matplotlib inline

Используя данные проведенного АB-теста, давайте проанализируем мощность теста. Ваше задание - при помощи библиотеки метода TTestIndPower оценить, какой размер выборки нам потребуется в дальнейшем если мы:

1. Хотим использовать t-test для сравнения CTR по юзерам.

2. Ожидаем, что размер эффекта равен 0.3 .

3. Мы хотим, чтобы мощность нашего теста равнялась 0.8.

Также рассчитайте, чему оказался равен размер эффекта в нашем проведенном АB-тесте, и сделайте вывод, могли бы использовать меньший размер выборки при сохранении мощности на уровне 0.8.

#### Оценим размер выборки, который нам потребуется в дальнейшем, для A/B тестирования со следующими параметрами

In [2]:
# Расчитаем необходимый размер выборки для следующих параметров
effect = 0.3
alpha = 0.05
power = 0.8
analysis = TTestIndPower()

# Оценим раззмер выборки
size = analysis.solve_power(effect, power=power, alpha=alpha)

print('Размер выборки, шт.: ', round(size))

Размер выборки, шт.:  175


#### Оценим размер эффекта для A/B теста проведенного с группами 0 и 1, в период с 2021-11-29 по 2021-12-05 включительно

In [3]:
# описываем соединение с БД
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'
}
# описываем sql запрос
q = """SELECT user_id, 
exp_group,
countIf(action='like') likes, 
countIf(action='view') views, 
likes / views ctr
FROM {db}.feed_actions 
where toDate(time) >= '2021-11-29' and toDate(time) <= '2021-12-05' and
exp_group in (0, 1)
group by user_id, exp_group
"""
# берем данные по sql запросу и кладем их df
df = pandahouse.read_clickhouse(q, connection=connection)
df.head()

,user_id,exp_group,likes,views,ctr
0,109963,1,4,13,0.307692
1,138232,1,41,166,0.246988
2,49204,1,39,134,0.291045
3,49286,1,6,61,0.098361
4,145957,1,46,202,0.227723


In [4]:
# Оценим размер выборки группы 0 и группы 1
count_group_0=df[df.exp_group == 0].ctr.count()
count_group_1=df[df.exp_group == 1].ctr.count()
print('Размер выборки 0: ', count_group_0)
print('Размер выборки 1: ', count_group_1)

Размер выборки 0:  12466
Размер выборки 1:  12558


In [5]:
# Расчитаем стандартне отклонения CTR для наших групп 
import statistics
std_dev_0 = statistics.stdev(df[df.exp_group == 0].ctr)
std_dev_1 = statistics.stdev(df[df.exp_group == 1].ctr)

In [6]:
# Расчитаем взвешенное стандартное отклонение для выборок (считаем что) одинакового размера
import math
s_pooled=math.sqrt((std_dev_0**(2)+std_dev_1**(2))/2)
s_pooled

0.08283403475781104

In [9]:
# Расчитаем размер эффекта
ES=round(abs((df[df.exp_group == 0].ctr.mean()-df[df.exp_group == 1].ctr.mean())/s_pooled),3)
ES

0.188

#### Расчитаем необходимый размер выборки для получения такого размера эффекта с мощностью 0.8 и порогом 0.05

In [13]:
size = analysis.solve_power(ES, power=0.8, alpha=0.05)
round(size)

445

#### Для порведения такого A/B теста с сохранением мощности 0.8 достаточно было бы по 445 измерений в каждой группе. 12к+ измерений цифра явно завышенная.